# Validation of job titles in bios

In [37]:
# Standard library imports
import sys

# Third party imports
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import prince
from networkx.algorithms import bipartite
from matplotlib.patches import Patch
import community as community_louvain
from netgraph import Graph


from nltk import ngrams
from collections import Counter

# Local application imports
sys.path.insert(0, '../Utility files')
import utils2
from utils2 import *


In [2]:

# load data
path = '/home/livtollanes/NewData/french/french1_updated.csv'

req_cols = ['follower_id', 'screen_name', 'description','description_cleantext','location', 'language', 'country']
dtypes = {'follower_id': 'object',
          'screen_name': 'object', 
          'description': 'object',
          'description_cleantext': 'object',
          'location': 'object',
          'language': 'object',
          'country': 'object'}


bios = pd.read_csv(path, usecols=req_cols, dtype=dtypes)

In [3]:
bios.head()

,follower_id,screen_name,description,location,description_cleantext,language,country
0,2419429508,Max_Mth_,"Radio, web, magazine - culture, société, vie a...","paris, france","Radio, web, magazine - culture, société, vie a...",fr,France
1,902419434358403072,Hellodie41,Ma thérapie : raconter ma vie a des gens qui n...,"blois, france",Ma thérapie : raconter ma vie a des gens qui n...,fr,France
2,44372703,julienmielcarek,Directeur des rédactions digitales BFM-RMC / C...,paris,Directeur des rédactions digitales BFM-RMC / C...,fr,France
3,1640041946,QDebrand,Directeur Adjoint @LDLC_Arena,"lyon, france",Directeur Adjoint @LDLC_Arena,fr,France
4,3118973117,jeanluc_bouchet,cadre informatique secteur hospitalier\nIntérê...,"betton, france",cadre informatique secteur hospitalierIntérêt ...,fr,France


the data below comes from https://www.insee.fr/fr/information/6450559

In [3]:
titles = pd.read_excel('/home/livtollanes/10.jan-thesis/labeldata/titles.xlsx')

In [8]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
titles


,title_man,title_female
0,Abat-jouriste,Abat-jouriste
1,Abatteur d'arbres,Abatteuse d'arbres
2,Abatteur de bestiaux,Abatteuse de bestiaux
3,Abatteur de carrières,Abatteuse de carrières
4,Abatteur de mines,Abatteuse de mines
5,Abouteur colleur,Abouteuse colleuse
6,Accessoiriste de décor,Accessoiriste de décor
7,Accessoiriste de plateau,Accessoiriste de plateau
8,Accompagnant éducatif et social (AES) à domicile,Accompagnante éducative et sociale (AES) à domicile
9,Accompagnant éducatif et social (AES) à l'éducation inclusive et à la vie ordinaire,Accompagnante éducative et sociale (AES) à l'éducation inclusive et à la vie ordinaire


In [5]:
words1 = titles['title_female'].tolist()
words2 = titles['title_man'].tolist()

# Convert all items in the list to strings
words1 = [str(item) for item in words1]
words2 = [str(item) for item in words2]

# Create a pattern for the words to be searched
pattern1 = '|'.join(words1)
pattern2 = '|'.join(words2)

# Use str.contains to get rows where 'description_cleantext' contains any word from the lists
filtered_df1 = bios[bios['description_cleantext'].str.contains(pattern1, na=False)]
filtered_df2 = bios[bios['description_cleantext'].str.contains(pattern2, na=False)]

/tmp/ipykernel_1559056/841559411.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df1 = bios[bios['description_cleantext'].str.contains(pattern1, na=False)]
/tmp/ipykernel_1559056/841559411.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df2 = bios[bios['description_cleantext'].str.contains(pattern2, na=False)]


In [12]:
def find_matching_word(description, words):
    for word in words:
        if contains_first_or_all_words(description, word):
            return word
    return None

bios['job1'] = bios['description_cleantext'].apply(lambda x: find_matching_word(x, words1))
bios['job2'] = bios['description_cleantext'].apply(lambda x: find_matching_word(x, words2))

# Combine 'job1' and 'job2' into a single 'job' column
bios['job'] = bios['job1'].combine_first(bios['job2'])

# Drop the 'job1' and 'job2' columns
bios.drop(['job1', 'job2'], axis=1, inplace=True)

In [18]:
bios_with_job = bios[bios['job'].notnull()]
bios_with_job[['job','description_cleantext']].iloc[30:50]

,job,description_cleantext
76,Régisseuse d'événementiel,Régisseuse d œuvres d art padawan féministe passionnée d arts
77,Journaliste,JournalisteRéd en chef RF magex lamanchelibreCollabore Figaro Magazine LeMeridionalOff et Asie Reportages
78,nan,membre titulaire de la commission éthique de nantesmetropole
80,Maire,Maire d AulnaySousBois Président de Paris terresdenvol Conseiller régional d iledefrance Secrétaire National lesrepublicains en charge de la Sécurité
85,nan,Pratique le culte du jus d ananas Instagram lolilas_
91,nan,Journaliste en formation à l EPJTours bretonnante Zellidja
92,Ébéniste,Sami Dubosc Ébéniste designer manufactureduboscetfils design art madeinfrance tih malvoyant handicap boosteurdebonheur les3b activateurdeprogrès
93,Social media manager,Bienvenue sur le compte Twitter des Socialistes de Montrouge social écologique solidaire jaimemontrouge montrougemaville
95,Journaliste,Journaliste reporter d images Ministère de l Éducation
101,Conseiller des activités physiques et sportives,Dircomcab externalisé ComPublique Compol Influence attractivité RP Conseiller Municipal Cassis Pdt think tank HELP Sec Nal E M Écologie Modernisme


In [8]:
import re

# Define a function that finds matching words in a string for words1
def find_matching_words1(description):
    matches = []
    for word in words1:
        # Split the word by commas and parentheses
        parts = re.split(',|\(|\)', word)
        # Check each part separately
        for part in parts:
            part = part.strip()  # Remove leading and trailing spaces
            if part and part in description:
                matches.append(part)
    return matches

# Apply the function to the 'description_cleantext' column and assign the result to a new column 'job1'
filtered_df1['job1'] = filtered_df1['description_cleantext'].apply(find_matching_words1)

# Define a function that finds matching words in a string for words2
def find_matching_words2(description):
    matches = []
    for word in words2:
        # Split the word by commas and parentheses
        parts = re.split(',|\(|\)', word)
        # Check each part separately
        for part in parts:
            part = part.strip()  # Remove leading and trailing spaces
            if part and part in description:
                matches.append(part)
    return matches

# Apply the function to the 'description_cleantext' column and assign the result to a new column 'job2'
filtered_df2['job2'] = filtered_df2['description_cleantext'].apply(find_matching_words2)

/tmp/ipykernel_1515732/1780242890.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['job1'] = filtered_df1['description_cleantext'].apply(find_matching_words1)
/tmp/ipykernel_1515732/1780242890.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2['job2'] = filtered_df2['description_cleantext'].apply(find_matching_words2)


In [20]:
# Apply the function to the 'description_cleantext' column and assign the result to a new column 'job2'
filtered_df1['job1'] = filtered_df2['description_cleantext'].apply(find_matching_words1)

/tmp/ipykernel_1515732/4228196134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['job1'] = filtered_df2['description_cleantext'].apply(find_matching_words1)


In [22]:
pd.set_option('display.max_colwidth', None)
filtered_df2[['description_cleantext', 'job2']].iloc[150:190]

,description_cleantext,job2
985,Suppléante du Député @T_Cazenave sur la #Gironde #Bordeaux #Bruges #LeBouscat #renaissance Responsable centre de formation,"[eau, eau, eau, Député, eau]"
986,"Maire adjointe @mairieivry en charge de la #culture, du #patrimoine, du #tourisme, de la #villenumérique, des #travaux et des affaires civiles et juridiques","[numérique, numérique, adjoint, numérique, Maire, Maire adjoint, Maire, Maire, adjoint, numérique, numérique, culture]"
999,Infirmier Libéral--NBA— #MileHighBasketball CinéSeries--Numerique--Design--Lyon mais Bressan,"[Infirmier, Lyon]"
1001,"La Confédération Nationale des Avocats, 1er syndicat historique d'avocats, regroupe tous les avocats soucieux des intérêts de la profession comme de la justice.",[Avocat]
1006,Chasseuse de tête - Consultante chez @CadrAvenir / ex-Journaliste @RTL et @LCI-TF1,[Journaliste]
1020,"Avocat. Je suis responsable de ce que je dis, pas de ce vous comprenez #Teamjuriste #Babylawyer",[Avocat]
1031,Journaliste @MidiLibrelozere Localier de métier. Breton de naissance. Napolitain d'adoption. Papa de Matei #ForzaNapoliSempre #СлаваУкраїні,[Journaliste]
1036,Président de IDA (Intégrer Développer Agir) Secrétaire général de APSM (Alliance Patriotique pour la Souveraineté du Mali) président d’Espérance Média MaliDjigu,"[Secrétaire, Secrétaire général, Secrétaire général]"
1038,Avocat en droit du travail. Du côté de la force.,[Avocat]
1044,Ancienne Députée de Loire-Atlantique Circo4404 Commission Aff. Étrangères I Assemblée Parlementaire de la #Francophonie I Droits de l’Homme I #JOP2024,"[Député, Parlementaire]"


In [15]:
# Create a mask for rows where 'job2' contains only one unique value
mask = filtered_df2['job2'].apply(lambda x: len(set(x)) == 1)

# Use the mask to filter the DataFrame
unique_values_df = filtered_df2[mask]

# Print the unique values
print(unique_values_df['job2'])

11                [Psychologue]
17                [Journaliste]
34                     [Député]
50                [Journaliste]
91                [Journaliste]
                  ...          
115020      [Rédacteur en chef]
115029                 [Député]
115030                [Juriste]
115056            [Chroniqueur]
115065    [Maire, Maire, Maire]
Name: job2, Length: 6527, dtype: object


In [17]:
# Get the counts of each unique value in 'job2'
counts = unique_values_df['job2'].value_counts()

# Print the counts
pd.set_option('display.max_rows', None)
print(counts)

job2
[Journaliste]                                     2777
[Maire, Maire, Maire]                              328
[Avocat]                                           244
[Photographe]                                      231
[Juriste]                                          192
[Député]                                           126
[Secrétaire]                                       115
[Acteur]                                           110
[Comédien]                                          92
[Rédacteur en chef]                                 86
[Graphiste]                                         77
[Écrivain]                                          58
[Infirmier]                                         52
[Blogueur]                                          47
[Responsable communication]                         47
[Musicien]                                          47
[Ambassadeur]                                       44
[Chroniqueur]                                       41
[Phar

In [20]:
import pandas as pd

data = {
    'Title': ['Marin-pêcheur', 'Marin-pêcheuse', 'Artisan', 'Artisane', 'Commerçant', 'Commerçante', 'Journaliste', 'Avocat', 'Avocate', 'Juriste', 'Pharmacien', 'Pharmacienne', 'Professeur d\'art', 'Professeure d\'arts', 'Chirurgien dentiste', 'Chirurgienne dentiste', 'Vétérinaire', 'Magistrat', 'Magistrate', 'Photographe', 'Infirmier', 'Infirmière', 'Sage-femme', 'Contrôleur des impôts', 'Opticien', 'Opticienne', 'Bibliothécaire', 'Puériculteur', 'Puéricultrice', 'Ambulancier', 'Ambulancière', 'Secrétaire', 'Caissier de magasin', 'Caissière de magasin', 'Concierge', 'Boulanger', 'Boulangère', 'Nettoyeur', 'Nettoyeuse', 'Jardinier', 'Jardinière', 'Ouvrier d\'art', 'Ouvrière d\'art', 'Docker', 'Diplomate', 'Ingénieur en chef', 'Médecin hospitalier'],
    'Income_monthly_av': [2400, 2400, 2300, 2300, 2600, 2600, 3500, 8000, 8000, 4100, 3600, 3600, 2900, 2900, 7000, 7000, 3300, 3600, 3600, 2100, 2600, 2600, 3100, 2600, 2600, 2600, 3100, 2800, 2800, 1800, 1800, 1900, 1600, 1600, 1800, 1800, 1800, 1500, 1500, 1600, 1600, 2200, 2200, 3700, 6500, 6500, 6000]
}

income = pd.DataFrame(data)

In [31]:
income.head()

,Title,Income_monthly_av
0,Marin-pêcheur,2400
1,Marin-pêcheuse,2400
2,Artisan,2300
3,Artisane,2300
4,Commerçant,2600


In [26]:
def find_job(description):
    words = description.lower().split()
    for title in income['Title']:
        if title.lower() in words:
            return title
    return None

bios['job'] = bios['description_cleantext'].apply(find_job)
jobs = bios.dropna(subset=['job'])

In [27]:
jobs[['description_cleantext', 'job']].iloc[30:50]

,description_cleantext,job
262,Franco québécoise Ex journaliste devenue dév Passionnée de justice sociale t1d type1 dt1,Journaliste
276,Adjointe Maire de Paris accessibilité universelle handicap Première secrétaire du PS_Paris Dr en pharmacie,Secrétaire
280,Juriste en droit des sociétés qui a toujours du temps pour dessiner lire découvrir des films et avec des copains c est encore mieux,Juriste
298,Journaliste politique écologie parfois critique cinéma à humanite_frTeam CallPolLeMedia Contact emilio meslet humanite fr,Journaliste
319,Journaliste,Journaliste
339,Journaliste à Presse Océan,Journaliste
351,Être ennemi des États Unis peut être dangereux mais être son ami est fatal Henry Kissinger secrétaire d état de Richard Nixon FREXIT,Secrétaire
354,Journaliste économique,Journaliste
365,Secrétaire National de la Défense du Parti Radical Si le radical c est l idéal oui je suis radical Victor Hugo,Secrétaire
369,Journaliste à lobs formée à EPJTours et ScPoLyon,Journaliste


In [29]:
jobs['job'].value_counts()

job
Journaliste       7106
Photographe        707
Secrétaire         640
Avocat             640
Juriste            469
Infirmière         230
Avocate            172
Infirmier          118
Artisan             96
Bibliothécaire      80
Pharmacien          71
Commerçant          32
Jardinier           31
Diplomate           26
Vétérinaire         23
Boulanger           19
Ambulancier         17
Magistrat           16
Opticien            15
Pharmacienne         8
Docker               7
Commerçante          7
Concierge            7
Magistrate           7
Jardinière           6
Puéricultrice        5
Opticienne           2
Artisane             2
Ambulancière         2
Boulangère           2
Name: count, dtype: int64

In [30]:
jobs.columns

Index(['follower_id', 'screen_name', 'description', 'location',
       'description_cleantext', 'language', 'country', 'job'],
      dtype='object')

In [32]:
jobs = jobs.merge(income, left_on='job', right_on='Title', how='left')

In [33]:
jobs.head()

,follower_id,screen_name,description,location,description_cleantext,language,country,job,Title,Income_monthly_av
0,15723458,cedric_o,Ex Secrétaire d’Etat chargé du numérique / Former Minister of State for Digital 🇫🇷🇪🇺,paris,Ex Secrétaire d Etat chargé du numérique Former Minister of State for Digital,fr,France,Secrétaire,Secrétaire,1900
1,196762472,ginolachat,Journaliste @courrierpicard Amiens.,amiens (80),Journaliste courrierpicard Amiens,fr,France,Journaliste,Journaliste,3500
2,3377827389,mat_monot,Conseiller dép. de la #SSD93 - Président de @SSDhabitat - 1er Adjoint de #Pantin - 1er Secrétaire Fédéral de @fedeps93 - Secrétaire natio à la Formation du PS,"pantin, france",Conseiller dép de la SSD93 Président de SSDhabitat 1er Adjoint de Pantin 1er Secrétaire Fédéral de fedeps93 Secrétaire natio à la Formation du PS,fr,France,Secrétaire,Secrétaire,1900
3,2809934157,PatSolange,"Journaliste/Présentatrice chez Africa24, rédactrice @aminamagazine #télé #infos #musique #cinéma #mode #voyages #tourisme #Antilles",france,Journaliste Présentatrice chez Africa24 rédactrice aminamagazine télé infos musique cinéma mode voyages tourisme Antilles,fr,France,Journaliste,Journaliste,3500
4,1637905050,Maud_Guilbeault,Journaliste médias @LaCroix / formée @EjToulouse,paris,Journaliste médias LaCroix formée EjToulouse,fr,France,Journaliste,Journaliste,3500


In [44]:
jobs.dtypes

follower_id              object
screen_name              object
description              object
location                 object
description_cleantext    object
language                 object
country                  object
job                      object
Title                    object
Income_monthly_av         int64
dtype: object

In [41]:
coords['follower_id'] = coords['follower_id'].astype('object')

In [45]:
coords.dtypes

follower_id     object
0              float64
1              float64
2              float64
3              float64
dtype: object

In [34]:
import pandas as pd

coords = pd.read_csv('/home/livtollanes/NewData/coordinates_labeled_subsets/m1_lab_coords/m1_lab_row_coordinates.csv')

In [47]:
jobs.head()

,follower_id,screen_name,description,location,description_cleantext,language,country,job,Title,Income_monthly_av
0,15723458,cedric_o,Ex Secrétaire d’Etat chargé du numérique / Former Minister of State for Digital 🇫🇷🇪🇺,paris,Ex Secrétaire d Etat chargé du numérique Former Minister of State for Digital,fr,France,Secrétaire,Secrétaire,1900
1,196762472,ginolachat,Journaliste @courrierpicard Amiens.,amiens (80),Journaliste courrierpicard Amiens,fr,France,Journaliste,Journaliste,3500
2,3377827389,mat_monot,Conseiller dép. de la #SSD93 - Président de @SSDhabitat - 1er Adjoint de #Pantin - 1er Secrétaire Fédéral de @fedeps93 - Secrétaire natio à la Formation du PS,"pantin, france",Conseiller dép de la SSD93 Président de SSDhabitat 1er Adjoint de Pantin 1er Secrétaire Fédéral de fedeps93 Secrétaire natio à la Formation du PS,fr,France,Secrétaire,Secrétaire,1900
3,2809934157,PatSolange,"Journaliste/Présentatrice chez Africa24, rédactrice @aminamagazine #télé #infos #musique #cinéma #mode #voyages #tourisme #Antilles",france,Journaliste Présentatrice chez Africa24 rédactrice aminamagazine télé infos musique cinéma mode voyages tourisme Antilles,fr,France,Journaliste,Journaliste,3500
4,1637905050,Maud_Guilbeault,Journaliste médias @LaCroix / formée @EjToulouse,paris,Journaliste médias LaCroix formée EjToulouse,fr,France,Journaliste,Journaliste,3500


In [43]:
coords_streamlined = utils2.streamline_IDs(source= jobs, df_tofilter= coords, column= 'follower_id')

Number of unique follower_id in source DataFrame: 10563
Number of unique follower_id in filtered DataFrame after filtering: 0
Removed 65745 rows from the DataFrame to be filtered.
0 rows are left in the filtered DataFrame.


In [48]:
coords.columns

Index(['follower_id', '0', '1', '2', '3'], dtype='object')

In [49]:
coords1 = coords.rename(columns={'0': 'd0'})
jobs1 = jobs.merge(coords1[['follower_id', 'd0']], on='follower_id', how='left')

In [51]:
coords['follower_id'].nunique()

65745

In [52]:
jobs['follower_id'].nunique()

10563